# Advent of Code 2023

## Day 03

https://adventofcode.com/2023/day/3

In [ ]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=3)

In [25]:
example_schematic = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

class Engine:
    schematic: list[list]
    
    def __init__(self, schematic_raw: str) -> None:
        self.schematic = [list(i) for i in schematic_raw.split('\n')]

    def __str__(self) -> str:
        return '\n'.join(''.join(i if i else '.' for i in row) for row in self.schematic)
    
print(Engine(example_schematic))

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
